<a href="https://colab.research.google.com/github/claudioEma/DistribuzioneFerie/blob/main/DistribuzioneFerie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 60.1 MB/s eta 0:00:00


In [3]:
from pulp import *

# Dati d'esempio
N = 5   # persone
T = 7   # giorni
M = 2   # gruppi

ferie_richieste = [2, 3, 2, 1, 2]     # giorni di ferie per ogni persona

preferenze = [                       # punteggio gradimento per ogni persona e giorno
    [10, 5, 0, 2, 4, 8, 6],          # persona 0
    [0, 3, 8, 6, 1, 2, 5],           # persona 1
    [2, 4, 6, 8, 10, 3, 1],          # persona 2
    [5, 2, 1, 7, 8, 6, 4],           # persona 3
    [6, 9, 4, 2, 3, 5, 7]            # persona 4
]

gruppi = [0, 0, 1, 1, 1]             # persona 0 e 1 nel gruppo 0, le altre nel gruppo 1

copertura_minima = [                # per ogni gruppo e giorno
    [1, 1, 1, 1, 1, 1, 1],           # gruppo 0
    [2, 2, 2, 2, 2, 2, 2]            # gruppo 1
]

# Modello
model = LpProblem("Pianificazione_Ferie", LpMaximize)

# Variabili binarie: x[i,d] = 1 se persona i ha ferie nel giorno d
x = LpVariable.dicts("x", [(i, d) for i in range(N) for d in range(T)], cat="Binary")

# Funzione obiettivo: massimizzare il gradimento complessivo
model += lpSum(preferenze[i][d] * x[(i, d)] for i in range(N) for d in range(T))

# Vincolo: ogni persona deve avere esattamente il numero di giorni di ferie richiesti
for i in range(N):
    model += lpSum(x[(i, d)] for d in range(T)) == ferie_richieste[i]

# Vincolo: copertura minima per gruppo per ogni giorno
for g in range(M):
    persone_nel_gruppo = [i for i in range(N) if gruppi[i] == g]
    for d in range(T):
        model += lpSum(x[(i, d)] for i in persone_nel_gruppo) <= len(persone_nel_gruppo) - copertura_minima[g][d]

# Risoluzione
model.solve()

# Output: ferie assegnate
for i in range(N):
    giorni = [d for d in range(T) if value(x[(i, d)]) == 1]
    print(f"Persona {i} (gruppo {gruppi[i]}) ha ferie nei giorni: {giorni}")


Persona 0 (gruppo 0) ha ferie nei giorni: [0, 5]
Persona 1 (gruppo 0) ha ferie nei giorni: [2, 3, 6]
Persona 2 (gruppo 1) ha ferie nei giorni: [3, 4]
Persona 3 (gruppo 1) ha ferie nei giorni: [5]
Persona 4 (gruppo 1) ha ferie nei giorni: [1, 6]


In [20]:
# Verifica vincolo: ogni persona deve avere esattamente il numero di giorni di
# ferie richiesti

for i in range(N):
  assert sum(x[(i, d)].varValue for d in range(T)) == ferie_richieste[i], f"La persona {i} ha {x[i,d].varValue} giorni di ferie invece di {ferie_richieste[i]}"

In [23]:
# Verifica vincolo: copertura minima per gruppo per ogni giorno

for g in range(M):
    persone_nel_gruppo = [i for i in range(N) if gruppi[i] == g]
    for d in range(T):
        inFerie = sum(x[(i, d)].varValue for i in persone_nel_gruppo) # numero di persone del gruppo g che sono in ferie il giorno d
        nonInFerie = len(persone_nel_gruppo) - inFerie # numero di persone del gruppo g che non sono in ferie il giorno d
        maxInFerie = len(persone_nel_gruppo) - copertura_minima[g][d] # massimo numero di persone del gruppo g che possono andare in ferie il giorno d
        assert inFerie <= maxInFerie, f"Il giorno {d} il gruppo {g} ha solo {nonInFerie} persone ma la copertura minima richiesta è {copertura_minima[g][d]} persone"


In [28]:
# Verifica qualità soluzione

maxConVincoli = sum(preferenze[i][d] * x[(i, d)].varValue for i in range(N) for d in range(T))

maxSenzaVincoli = 0
for i in range(N):
  maxSenzaVincoli += sum(sorted(preferenze[i], reverse=True)[:ferie_richieste[i]])

print(f"Valore della soluzione trovata: {maxConVincoli}")
print(f"Valore massimo ottenibile rimuovendo il vincolo della copertura: {maxSenzaVincoli}")
print(f"Gradimento generale soluzione: {maxConVincoli/maxSenzaVincoli:.3f}")

Valore della soluzione trovata: 77.0
Valore massimo ottenibile rimuovendo il vincolo della copertura: 79
Gradimento generale soluzione: 0.975
